# Итоговое задание №2. Прогнозирование конечных свойств новых материалов (композиционных материалов)
На входе имеются данные о начальных свойствах компонентов композиционных материалов (количество связующего, наполнителя, температурный режим отверждения и т.д.). На выходе необходимо спрогнозировать ряд конечных свойств получаемых композиционных материалов. 

Предсказание (из семинара 22): 
* модуль упругости при растяжении
* прочности при растяжении
* cоотношение матрица-наполнитель

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sl

In [4]:
df = pd.read_csv('../data/interim/df.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла
df.head()

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2","Угол нашивки, град",Шаг нашивки,Плотность нашивки
0,1.857143,2030.0,738.736842,30.00,22.267857,100.000000,210.0,70.0,3000.0,220.0,0,4.0,57.0
1,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0,0,4.0,60.0
2,1.857143,2030.0,738.736842,49.90,33.000000,284.615385,210.0,70.0,3000.0,220.0,0,4.0,70.0
3,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0,0,5.0,47.0
4,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,57.0


In [14]:
y_column_names = ['Модуль упругости при растяжении, ГПа','Прочность при растяжении, МПа','Соотношение матрица-наполнитель']
x_column_names = ['Плотность, кг/м3', 'модуль упругости, ГПа','Количество отвердителя, м.%','Содержание эпоксидных групп,%_2','Температура вспышки, С_2','Поверхностная плотность, г/м2','Потребление смолы, г/м2','Угол нашивки, град','Шаг нашивки','Плотность нашивки']

dfX = df.drop(columns=y_column_names)
dfY = df.drop(columns=x_column_names)

,Соотношение матрица-наполнитель,"Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа"
0,1.857143,70.0,3000.0
1,1.857143,70.0,3000.0
2,1.857143,70.0,3000.0
3,1.857143,70.0,3000.0
4,2.771331,70.0,3000.0


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(dfX.values,
                                                      dfY.values, test_size=.3, random_state=17)

print(f'Количество объектов в тренировочных данных: {X_train.shape[0]}')
print(f'Количество объектов в валидационных данных: {X_valid.shape[0]}')                                                      

Количество объектов в тренировочных данных: 716
Количество объектов в валидационных данных: 307


In [16]:

from keras.models import Sequential
from keras.layers import Dense

ModuleNotFoundError: No module named 'tensorflow'